In [5]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
import plotly.express as px
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from   harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy


In [7]:
# files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035*.nc')
files = glob.glob('/raid60/scratch/uli/0-20000-0-93*.nc')
files[0]

'/raid60/scratch/uli/0-20000-0-93062_CEUAS_merged_v0.nc'

In [7]:
def do_resort(fn):
    
    data = eua.CDMDataset(fn)

    allvars = data.observations_table.observed_variable[()]
    allvars.sort()
    allvars = numpy.unique(allvars)
    #
    ri = data.recordindex[()]
    print('recordindex: ', len(ri))
    rt = data.recordtimestamp[()]
    keys = data.observations_table.keys()[:-1]
    # dropping all keys, where dimensions won't work - just help variabels for dimensions
    keys.pop(43)
    keys.pop(42)
    keys.pop(41)
    recordindices = [[] for i in range(len(allvars))]
    recordtimestamps = [[] for i in range(len(allvars))]

    # output variables (from observations_table)
    ov = []
    for o in keys:
        ov.append([[] for i in range(len(numpy.unique(allvars)))])

    #
    # load data into memory and decode byte arrays
    #
    print('loading data')
    obsv = data.observations_table.observed_variable[:]
    ov_vars = []
    for o in range(len(keys)):
        ov_vars.append(data.observations_table[keys[o]][:])

    #
    # resorting the data
    #
    print('resort:start')
    @njit
    def make_vrindex(vridx,ridx,idx):
        l=0
        for i in range(1,len(idx)): # to set the recordindices
            if ridx[i]>ridx[i-1]:
                vridx[ridx[i-1]:ridx[i]]=l # next record after l
                l=i
        vridx[ridx[i]:]=len(idx) # next record for the last element is the len of the data


    tt=time.time()

    ridxall=np.zeros(ov_vars[0].shape[0],dtype=np.int64) # reverse index - index of the record index
    j=-1
    for j in range(len(ri)-1):
        ridxall[ri[j]:ri[j+1]]=j
    j+=1
    ridxall[ri[j]:]=j # for the last elemenet

    ridx=[]
    vridx=[]
    absidx=[]
    abscount=0
    for j in range(len(allvars)):
        idx=np.where(ov_vars[keys.index('observed_variable')]==allvars[j])[0] # index of all elements form certain variable j
        print(j,len(idx),',',end='')
        vridx.append(np.zeros(ri.shape[0],dtype=np.int64)) # all zeros in lenght of record index
        ridx=ridxall[idx] # ridxall where variable is j
        make_vrindex(vridx[-1],ridx,idx)
        vridx[-1]+=abscount # abscount for stacking the recordindex

        absidx.append(copy.copy(idx)) # why copy? - to make sure it's not just the ref. - maybe ok without the cp
        abscount+=len(idx)

    print('')
    absidx=np.concatenate(absidx)
    for o in range(len(keys)):
        ov_vars[o]=ov_vars[o][absidx]
        print(o,end='')
    #
    # recordtimestamps are only necessary once
    #
    recordtimestamps = recordtimestamps[0]


    out = ov_vars
    # targetfile has to be a copy of the original file
    targetfile = '/raid60/scratch/leo/scratch/'+fn.split('/')[-1]# 0-20000-0-63894_CEUAS_merged_v0.nc'
    if os.path.isfile(targetfile):
        mode='r+'
    else:
        mode='w'
        
    print()
    print('writing '+targetfile)
    #
    # writing data into observations_table
    #
    with h5py.File(targetfile, mode) as file:
        for i in range(len(keys)):
            try:
                del file['observations_table'][keys[i]]
            except:
                pass
    for i in range(len(keys)):
        if keys[i] == 'index':
            pass
        elif keys[i] == 'observation_id' or keys[i] == 'report_id' or keys[i] == 'sensor_id' or keys[i] == 'source_id':
#             slen = len(out[i][0])
            alldict = {keys[i]:np.asarray(out[i], dtype='S1')}
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])
        else:
            alldict = pandas.DataFrame({keys[i]:out[i]})
            write_dict_h5(targetfile, alldict, 'observations_table', {keys[i]: { 'compression': 'gzip' } }, [keys[i]])  
        #
        # writing the recordindices and recordtimestamp.
        #       
    recordindices=vridx
    with h5py.File(targetfile, 'r+') as file:
        for i in range(len(recordindices)):
            try:
                del file['recordindices'][str(allvars[i])]
            except:
                pass
    for i in range(len(recordindices)):
        testvar = pandas.DataFrame({str(allvars[i]):recordindices[i]})
        write_dict_h5(targetfile, testvar, 'recordindices', {str(allvars[i]): { 'compression': None } }, [str(allvars[i])]) 

    with h5py.File(targetfile, 'r+') as file:
        try:
            del file['recordindex']
        except:
            pass
        try:
            del file['recordindices']['recordtimestamp']
        except:
            pass
    write_dict_h5(targetfile, {'recordtimestamp':rt}, 'recordindices', {'recordtimestamp': { 'compression': None } }, ['recordtimestamp']) 
        
    print('elapsed:',time.time()-tt)
    print(end='')



# In[15]:




['/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11610_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11103_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11501_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11601_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11602_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11604_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11605_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11606_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11608_CEUAS_merged_v0.nc', '/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11615_CEUAS_merged_v0.nc', '/raid60/scratch/fe

recordindex:  9752
loading data
resort:start
0 201 1542 584663 500744 500745 58257
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11610_CEUAS_merged_v0.nc
elapsed: 0.9017224311828613
recordindex:  1257
loading data
resort:start
0 130731 13073
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11103_CEUAS_merged_v0.nc
elapsed: 1.517674207687378
recordindex:  1192
loading data
resort:start
0 138331 13833
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11501_CEUAS_merged_v0.nc
elapsed: 1.3312656879425049
recordindex:  3411
loading data
resort:start
0 636101 63610
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11601_CEUAS_merged_v0.nc
elapsed: 0.9509546756744385
record

elapsed: 0.7455244064331055
recordindex:  4555
loading data
resort:start
0 1446671 30192 1509033 1308964 1308965 95157
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11722_CEUAS_merged_v0.nc
elapsed: 2.5220327377319336
recordindex:  9
loading data
resort:start
0 141 142 173 144 6
012345678910111213141516171819202122232425262728293031323334353637383940414243444546writing /raid60/scratch/leo/scratch/0-20000-0-11024_CEUAS_merged_v0.nc
elapsed: 0.9576425552368164
recordindex:  1
loading data
resort:start


IndexError: index 5 is out of bounds for axis 0 with size 1

In [ ]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-1????_*.nc')
print(files)
tt=time.time()
p=Pool(20)
p.map(do_resort,files)


In [15]:
files = glob.glob('/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-1????_*.nc')
files[0]

'/raid60/scratch/federico/MERGED_DATABASE_OCTOBER2020_sensor/0-20000-0-11035_CEUAS_merged_v0.nc'

In [35]:
with eua.CDMDataset(targetfile) as data_test:
    print(data_test)

File: <HDF5 file "0-20000-0-11035_CEUAS_merged_v0.nc" (mode r)>
Filename: /raid60/scratch/uli/testdata/0-20000-0-11035_CEUAS_merged_v0.nc
(G)roups/(V)ariables: 

 - G | crs__________________________________________ : : 4
 - V | dateindex____________________________________ : : (26625,)
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 49
 - G | observed_variable____________________________ : : 9
 - G | recordindices________________________________ : : 17
 - V | recordtimestamp______________________________ : : (91456,)
 - G | sensor_configuration_________________________ : : 12
 - G | source_configuration_________________________ : : 2
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 7
 - G | station_type_________________________________ : : 4
 - G | units_______________________________________

In [28]:
data_test.observations_table

observations_table:

adjustment_id_____________________________________ : : (34967495,)
advanced_assimilation_feedback____________________ : : (34967495,)
advanced_homogenisation___________________________ : : (34967495,)
advanced_qc_______________________________________ : : (34967495,)
advanced_uncertainty______________________________ : : (34967495,)
bbox_max_latitude_________________________________ : : (34967495,)
bbox_max_longitude________________________________ : : (34967495,)
bbox_min_latitude_________________________________ : : (34967495,)
bbox_min_longitude________________________________ : : (34967495,)
code_table________________________________________ : : (34967495,)
conversion_flag___________________________________ : : (34967495,)
conversion_method_________________________________ : : (34967495,)
crs_______________________________________________ : : (34967495,)
data_policy_licence_______________________________ : : (34967495,)
date_time________________________________

In [29]:
data_test.recordindices

recordindices:

0_________________________________________________ : : (91456,)
104_______________________________________________ : : (91456,)
105_______________________________________________ : : (91456,)
106_______________________________________________ : : (91456,)
107_______________________________________________ : : (91456,)
117_______________________________________________ : : (91456,)
136_______________________________________________ : : (91456,)
137_______________________________________________ : : (91456,)
138_______________________________________________ : : (91456,)
139_______________________________________________ : : (91456,)
140_______________________________________________ : : (91456,)
34________________________________________________ : : (91456,)
36________________________________________________ : : (91456,)
38________________________________________________ : : (91456,)
39________________________________________________ : : (91456,)
85______________________

---

In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# # recordindices einzeln sortieren - für alle variablen eigene listen erstellen - dazu recordindices erstellen - alle daten hintereinander legen - len aller vorhergehender rec_in zu den nachkommenden addieren.
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# variables = [[] for i in range(len(np.unique(allvars)))]
# values = [[] for i in range(len(np.unique(allvars)))]
# zcoords = [[] for i in range(len(np.unique(allvars)))]

# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     b = data.observations_table.observation_value[()][start:end]
#     c = data.observations_table.z_coordinate[()][start:end]

#     sa, sb = zip(*sorted(zip(a, b)))
#     sa, sc = zip(*sorted(zip(a, c)))
#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 variables[j].append(sa[k])
#                 values[j].append(sb[k])
#                 zcoords[j].append(sc[k])
#         recordindices[j].append(len(variables[j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# recordindices = [[] for i in range(len(np.unique(allvars)))]
# recordtimestamps = [[] for i in range(len(np.unique(allvars)))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])
    
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         break
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = data.observations_table[keys[o]][()][start:end]
#         try: # all varibales with different shape won't work here
#             sa, sb = zip(*sorted(zip(a, b)))
#             helpvar.append(sb)
#         except:
#             pass

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     try: # all varibales with different shape won't work here
#                         ov[m][j].append(helpvar[m][k])
#                     except:
#                         pass
#         recordindices[j].append(len(ov[0][j]))
#         recordtimestamps[j].append(rt[i])
        
# lenadd = 0
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(variables[i])


In [ ]:
# #
# #
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print(len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])    
    
# print('resort:start')
# for i in range(len(ri)):
#     print(i)
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = data.observations_table.observed_variable[()][start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#             b = []
#             for n in data.observations_table[keys[o]][()][start:end]:
#                 c = ''
#                 for bb in n:
#                     c = c + bb.decode()
#                 b.append(c)
#         else:
#             b = data.observations_table[keys[o]][()][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# #
# for o in range(len(keys)):
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in range(len(out[o])):
#             c = []
#             for bb in out[o][n]:
#                 c.append(bb.encode())
#             out[o][n] = c


In [ ]:
# allvars = data.observations_table.observed_variable[()]
# allvars.sort()
# allvars = np.unique(allvars)
# #
# ri = data.recordindex[()]
# print('recordindex: ', len(ri))
# # rt = data.recordtimestamp[()]
# keys = data.observations_table.keys()[:-1]
# # dropping all keys, where dimensions won't work - just help variabels for dimensions
# keys.pop(43)
# keys.pop(42)
# keys.pop(41)
# recordindices = [[] for i in range(len(allvars))]
# recordtimestamps = [[] for i in range(len(allvars))]

# # output variables (from observations_table)
# ov = []
# for o in keys:
#     ov.append([[] for i in range(len(np.unique(allvars)))])

# #
# # load data into memory and decode byte arrays
# #
# print('loading data')
# obsv = data.observations_table.observed_variable[:]
# ov_vars = []
# for o in range(len(keys)):
#     ov_vars.append(data.observations_table[keys[o]][:])
#     if keys[o] == 'observation_id' or keys[o] == 'report_id' or keys[o] == 'sensor_id' or keys[o] == 'source_id':
#         b = []
#         for n in ov_vars[o]:
#             c = ''
#             for bb in n:
#                 c = c + bb.decode()
#             b.append(c)
#         ov_vars[o] = b

# #
# # resorting the data
# #
# print('resort:start')
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(data.observations_table.observed_variable[()])
        
#     a = obsv[start:end]
#     helpvar = []
#     for o in range(len(keys)):
#         b = ov_vars[o][start:end]
#         sa, sb = zip(*sorted(zip(a, b)))
#         helpvar.append(sb)

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))
# #         recordtimestamps[j].append(rt[i])

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# print('shaping record_indices')
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if recordindices[i][j] != 0 and recordindices[i][j] == recordindices[i][j-1]:
#             old = recordindices[i][j]
#             recordindices[i][j] = np.nan
#         elif recordindices[i][j] == old:
#             recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# print('stacking output variables')
# out = []
# for j in ov:
#     finalvar = []
#     for i in j:
#         finalvar = finalvar + i
#     out.append(finalvar)
    
# #
# # restoring byte arrays:
# # this takes very long -> find faster option
# #
# print('restoring byte arrays')
# bytelist = [25, 34, 38, 39]
# for o in bytelist:
#     originallen = len(data.observations_table[keys[o]][()][0])
#     for n in range(len(out[o])):
#         c = [elem.encode() for elem in out[o][n]]
#         # problem: if the string was [b'x', b'y', b''] in the first place, it get converted to 'xy' and then back to [b'x', b'y']
#         # add empty byte strings until the data is as long as before:
#         while(len(c) < originallen):
#             c.append(str('').encode())
#         out[o][n] = c

In [ ]:
# with eua.CDMDataset(files[0]) as data:
#     allvars = data.observations_table.observed_variable[()]
#     allvars.sort()
#     allvars = np.unique(allvars)
#     #
#     ri = data.recordindex[()]
#     print('recordindex: ', len(ri))
#     keys = data.observations_table.keys()[:-1]
#     # dropping all keys, where dimensions won't work - just help variabels for dimensions
#     keys.pop(43)
#     keys.pop(42)
#     keys.pop(41)
#     recordindices = [[] for i in range(len(allvars))]
#     recordtimestamps = [[] for i in range(len(allvars))]

#     # output variables (from observations_table)
#     ov = []
#     for o in keys:
#         ov.append([[] for i in range(len(np.unique(allvars)))])

#     #
#     # load data into memory and decode byte arrays
#     #
#     print('loading data')
#     obsv = data.observations_table.observed_variable[:]
#     ov_vars = []
#     for o in range(len(keys)):
#         ov_vars.append(data.observations_table[keys[o]][:])

# #
# # resorting the data
# #
# print('resort:start')
# for i in range(len(ri)):
#     try:
#         start = ri[i]
#         end = ri[i+1]
#     except:
#         start = ri[i]
#         end = len(obsv)
        
#     a = obsv[start:end]
#     sa, sortindex = zip(*sorted(zip(a, range(len(a)))))
#     sortindex = np.array(sortindex)
#     helpvar = []
#     for o in range(len(keys)):
#         b = ov_vars[o][start:end]
#         helpvar.append(b[sortindex])

#     for j in range(len(allvars)):
#         for k in range(len(sa)):
#             if sa[k] == allvars[j]:
#                 for m in range(len(helpvar)):
#                     ov[m][j].append(helpvar[m][k])
#         recordindices[j].append(len(ov[0][j]))

# print('resort:done')
        
# #
# # setting record_indices to the right value -> stacking them
# #
# lenadd = 0.
# for i in range(len(recordindices)):
#     recordindices[i] = np.asarray(np.append([0], recordindices[i][:-1])) + lenadd
#     lenadd += len(ov[0][i])

# #
# # shaping record_indices -> setting every missing value to nan
# #
# print('shaping record_indices')
# old = -1
# for i in range(len(recordindices)):
#     for j in range(len(recordindices[i])):
#         if i != 0 or j != 0:
#             if recordindices[i][j] == recordindices[i][j-1]:
#                 old = recordindices[i][j]
#                 recordindices[i][j] = np.nan
#             elif recordindices[i][j] == old:
#                 recordindices[i][j] = np.nan

# #
# # recordtimestamps are only necessary once
# #
# recordtimestamps = recordtimestamps[0]

# # 
# # stacking all output variables
# #
# print('stacking output variables')
# out = []
# for k in ov:
#     out.append([j for i in k for j in i])